In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")

In [2]:
train_data["Date"] = pd.to_datetime(train_data.Date,format="%Y-%m-%d")
test_data["Date"] = pd.to_datetime(test_data.Date,format="%Y-%m-%d")

In [3]:
# train_data["weekend"] = train_data["weekend"].astype("str")
# train_data["dayofweek"] = train_data["dayofweek"].astype("str")

In [4]:
train_data.dtypes

ID                       object
Store_id                  int64
Store_Type               object
Location_Type            object
Region_Code              object
Date             datetime64[ns]
Holiday                   int64
Discount                 object
#Order                    int64
Sales                   float64
dtype: object

In [5]:
train_data.set_index("ID",inplace=True)
test_data.set_index("ID",inplace=True)

bool_int_map={True:1, False:0}

# #Order column is missing in the input test data. Hence this data will be missing
train_data["Date"] = pd.to_datetime(train_data.Date,format="%Y-%m-%d")
test_data["Date"] = pd.to_datetime(test_data.Date,format="%Y-%m-%d")

# Is the day a weekend?
train_data["dayofweek"]=train_data["Date"].dt.dayofweek
test_data["dayofweek"]=test_data["Date"].dt.dayofweek
train_data["weekend"] = train_data["dayofweek"]>=5
test_data["weekend"] = test_data["dayofweek"]>=5
train_data.drop('dayofweek', inplace=True, axis=1)
test_data.drop('dayofweek', inplace=True, axis=1)

train_data["weekend"] = train_data["weekend"].map(bool_int_map)
test_data["weekend"] = test_data["weekend"].map(bool_int_map)

# train_data["quarter"]=train_data["Date"].dt.quarter
# test_data["quarter"]=test_data["Date"].dt.quarter

# train_data["Date"] = train_data["Date"].dt.date
# test_data["Date"] = test_data["Date"].dt.date
# # holiday_map = {0:False, 1:True}
# train_data["Holiday"] = train_data["Holiday"].map(holiday_map)
# test_data["Holiday"] = test_data["Holiday"].map(holiday_map)

discount_map = {"Yes":1, "No":0}
train_data["Discount"] = train_data["Discount"].map(discount_map)
test_data["Discount"] = test_data["Discount"].map(discount_map)

categ_fts =[ 'Store_Type', 'Location_Type', 'Region_Code']
for ft in categ_fts:
    train_data[ft] = train_data[ft].astype("category")
    test_data[ft] = test_data[ft].astype("category")


In [6]:
# int_bool_map = {0:False, 1:True}

# train_data["Holiday"] = train_data["Holiday"].map(int_bool_map)
# train_data["Discount"] = train_data["Discount"].map(int_bool_map)
train_data

,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales,weekend
ID,,,,,,,,,,
T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,0
T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,0
T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,0
T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,0
T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,0
...,...,...,...,...,...,...,...,...,...,...
T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,0
T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,0
T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,0


In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

num_fts = [x for x in test_data.select_dtypes(include="number").columns]
cat_fts = [x for x in test_data.select_dtypes(include=["bool","category"]).columns if x !="Store_id"]


target="Sales"
features=[x for x in test_data.columns if x not in target]

In [8]:
features
# train_data["Holiday"] = train_data["Holiday"].map(bool_int_map)
# train_data["Discount"] = train_data["Discount"].map(bool_int_map)

['Store_id',
 'Store_Type',
 'Location_Type',
 'Region_Code',
 'Date',
 'Holiday',
 'Discount',
 'weekend']

In [9]:
for ft in cat_fts:
    dummy = pd.get_dummies(train_data[ft], prefix=f"{ft[0]}{ft[ft.find('_')+1]}", drop_first=True)
    train_data = pd.merge(train_data, dummy, left_index=True, right_index=True).drop(ft,axis=1)

In [10]:
for ft in cat_fts:
    dummy = pd.get_dummies(test_data[ft], prefix=f"{ft[0]}{ft[ft.find('_')+1]}", drop_first=True)
    test_data = pd.merge(test_data, dummy, left_index=True, right_index=True).drop(ft,axis=1)

In [11]:
features = [x for x in test_data.select_dtypes("number").columns]

In [12]:
test_data.select_dtypes("number").columns

Index(['Store_id', 'Holiday', 'Discount', 'weekend', 'ST_S2', 'ST_S3', 'ST_S4',
       'LT_L2', 'LT_L3', 'LT_L4', 'LT_L5', 'RC_R2', 'RC_R3', 'RC_R4'],
      dtype='object')

In [13]:
dy2 = train_data[(train_data["Sales"]<200000)]
dy = train_data[(train_data["Sales"] <150000)]

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler

In [15]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,BaggingRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import HuberRegressor, LinearRegression

In [16]:
s_scaler = StandardScaler()
# for ft in features:
#     s_scaler.fit(train_data[ft].values.reshape(-1,1))
#     train_data[ft] = s_scaler.transform(train_data[ft].values.reshape(-1,1))
#     test_data[ft] = s_scaler.transform(test_data[ft].values.reshape(-1,1))

In [17]:
# sns.histplot(train_data)

In [18]:
train_data

,Store_id,Date,Holiday,Discount,#Order,Sales,weekend,ST_S2,ST_S3,ST_S4,LT_L2,LT_L3,LT_L4,LT_L5,RC_R2,RC_R3,RC_R4
ID,,,,,,,,,,,,,,,,,
T1000001,1,2018-01-01,1,1,9,7011.84,0,0,0,0,0,1,0,0,0,0,0
T1000002,253,2018-01-01,1,1,60,51789.12,0,0,0,1,1,0,0,0,0,0,0
T1000003,252,2018-01-01,1,1,42,36868.20,0,0,1,0,1,0,0,0,0,0,0
T1000004,251,2018-01-01,1,1,23,19715.16,0,1,0,0,0,1,0,0,0,0,0
T1000005,250,2018-01-01,1,1,62,45614.52,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1188336,149,2019-05-31,1,1,51,37272.00,0,1,0,0,0,1,0,0,1,0,0
T1188337,153,2019-05-31,1,0,90,54572.64,0,0,0,1,1,0,0,0,0,0,0
T1188338,154,2019-05-31,1,0,56,31624.56,0,0,0,0,0,1,0,0,1,0,0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size = 0.05)
models=[DecisionTreeRegressor, RandomForestRegressor]
for model in models:
    print(model)
    model_instance = model()
    model_instance.fit(X_train, y_train)
    y_hat = model_instance.predict(X_test)
    print(f"test_error: {mse(y_true=y_test, y_pred=y_hat)}")
    print(f"train_error: {mse(y_true=y_train, y_pred=model_instance.predict(X_train))}")
#     X_test[target]=model_instance.predict(X_test)
#     break

<class 'sklearn.tree._classes.DecisionTreeRegressor'>
test_error: 107962672.47528173
train_error: 103833624.69367427
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
test_error: 107942570.74206777
train_error: 103861113.74954541


In [20]:
# weekend and weekday info


<class 'sklearn.tree._classes.DecisionTreeRegressor'>
test_error: 130746800.6851016
train_error: 108954910.33079284
<class 'sklearn.tree._classes.DecisionTreeRegressor'>
test_error: 108789740.10433578
train_error: 103783211.1154453
<class 'sklearn.ensemble._forest.RandomForestRegressor'>
test_error: 108814290.20272683
train_error: 103807666.99687739

In [21]:
train_data.Date

ID
T1000001   2018-01-01
T1000002   2018-01-01
T1000003   2018-01-01
T1000004   2018-01-01
T1000005   2018-01-01
              ...    
T1188336   2019-05-31
T1188337   2019-05-31
T1188338   2019-05-31
T1188339   2019-05-31
T1188340   2019-05-31
Name: Date, Length: 188340, dtype: datetime64[ns]

In [22]:
train_data

,Store_id,Date,Holiday,Discount,#Order,Sales,weekend,ST_S2,ST_S3,ST_S4,LT_L2,LT_L3,LT_L4,LT_L5,RC_R2,RC_R3,RC_R4
ID,,,,,,,,,,,,,,,,,
T1000001,1,2018-01-01,1,1,9,7011.84,0,0,0,0,0,1,0,0,0,0,0
T1000002,253,2018-01-01,1,1,60,51789.12,0,0,0,1,1,0,0,0,0,0,0
T1000003,252,2018-01-01,1,1,42,36868.20,0,0,1,0,1,0,0,0,0,0,0
T1000004,251,2018-01-01,1,1,23,19715.16,0,1,0,0,0,1,0,0,0,0,0
T1000005,250,2018-01-01,1,1,62,45614.52,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1188336,149,2019-05-31,1,1,51,37272.00,0,1,0,0,0,1,0,0,1,0,0
T1188337,153,2019-05-31,1,0,90,54572.64,0,0,0,1,1,0,0,0,0,0,0
T1188338,154,2019-05-31,1,0,56,31624.56,0,0,0,0,0,1,0,0,1,0,0


In [23]:
model_instance = DecisionTreeRegressor()
# model_instance = RandomForestRegressor()
model_instance.fit(train_data[features], train_data[target])
test_data[f"{target}"] = model_instance.predict(test_data[features])
test_data[target].to_csv("submission_DTRegressor.csv")
# test_data[target].to_csv("submission_RFRegressor.csv")

In [24]:
train_data[features]

,Store_id,Holiday,Discount,weekend,ST_S2,ST_S3,ST_S4,LT_L2,LT_L3,LT_L4,LT_L5,RC_R2,RC_R3,RC_R4
ID,,,,,,,,,,,,,,
T1000001,1,1,1,0,0,0,0,0,1,0,0,0,0,0
T1000002,253,1,1,0,0,0,1,1,0,0,0,0,0,0
T1000003,252,1,1,0,0,1,0,1,0,0,0,0,0,0
T1000004,251,1,1,0,1,0,0,0,1,0,0,0,0,0
T1000005,250,1,1,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1188336,149,1,1,0,1,0,0,0,1,0,0,1,0,0
T1188337,153,1,0,0,0,0,1,1,0,0,0,0,0,0
T1188338,154,1,0,0,0,0,0,0,1,0,0,1,0,0


In [25]:
import xgboost as xgb
from xgboost import XGBRegressor
regressor = XGBRegressor()
dtrain = xgb.DMatrix(train_data[features])

In [26]:
model = XGBRegressor(
    max_depth=10,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,
    importance_type='gini',
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    verbose=True, 
    early_stopping_rounds = 2)

# rcParams['figure.figsize'] = 12, 4

[0]	validation_0-rmse:42248.24609	validation_1-rmse:42365.12109
[1]	validation_0-rmse:38351.94922	validation_1-rmse:38468.86328
[2]	validation_0-rmse:34978.71875	validation_1-rmse:35087.08984
[3]	validation_0-rmse:31848.13672	validation_1-rmse:31944.71680
[4]	validation_0-rmse:29093.86328	validation_1-rmse:29188.63281
[5]	validation_0-rmse:26676.71875	validation_1-rmse:26754.04688
[6]	validation_0-rmse:24544.88672	validation_1-rmse:24606.81641
[7]	validation_0-rmse:22631.93555	validation_1-rmse:22694.71680
[8]	validation_0-rmse:20918.11328	validation_1-rmse:20976.19531
[9]	validation_0-rmse:19406.23047	validation_1-rmse:19461.92969
[10]	validation_0-rmse:18122.98438	validation_1-rmse:18182.72852
[11]	validation_0-rmse:16966.63086	validation_1-rmse:17026.09961
[12]	validation_0-rmse:15965.71191	validation_1-rmse:16024.30078
[13]	validation_0-rmse:15104.75781	validation_1-rmse:15164.23242
[14]	validation_0-rmse:14358.17481	validation_1-rmse:14417.40723
[15]	validation_0-rmse:13728.65625	

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gini', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=10,
             min_child_weight=0.5, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=0.8, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [27]:
# Further optimizing this model resulted in deteoriation of rank on public leader dashboard.
# validation_0-rmse:10214.00488	validation_1-rmse:10396.72754 
# validation_0-rmse:9834.18750	validation_1-rmse:10185.91602
# validation_0-rmse:9854.59570	validation_1-rmse:10168.09473
# Hence reverting back 

In [28]:
test_data[target] =  model.predict(test_data[features])

In [29]:
test_data

,Store_id,Date,Holiday,Discount,weekend,ST_S2,ST_S3,ST_S4,LT_L2,LT_L3,LT_L4,LT_L5,RC_R2,RC_R3,RC_R4,Sales__,Sales
ID,,,,,,,,,,,,,,,,,
T1188341,171,2019-06-01,0,0,1,0,0,1,1,0,0,0,0,1,0,54703.869565,54773.941406
T1188342,172,2019-06-01,0,0,1,0,0,0,0,0,0,0,0,0,0,39991.391304,39733.554688
T1188343,173,2019-06-01,0,0,1,0,0,1,1,0,0,0,0,0,0,79010.528571,78461.703125
T1188344,174,2019-06-01,0,0,1,0,0,0,0,0,0,0,0,0,1,36995.442857,38655.078125
T1188345,170,2019-06-01,0,0,1,0,0,0,0,0,0,0,1,0,0,40285.628571,40455.281250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1210601,186,2019-07-31,0,0,0,1,0,0,0,0,0,1,1,0,0,19050.291892,19224.312500
T1210602,11,2019-07-31,0,0,0,0,0,1,1,0,0,0,0,0,0,54123.813559,54135.664062
T1210603,185,2019-07-31,0,1,0,0,0,0,0,0,0,0,0,1,0,46290.064286,46121.074219


In [30]:
# test_data

In [31]:
test_data[target].to_csv("submission_XGBRegressor.csv")